<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/python/WORK%20for%20P02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WORK for P02

## [Work-1]  
以下の設問に答えて下さい。
###### Q1.リッジ回帰とはどのようなアルゴリズムか（OLSとの違いは）？  
①OLSにL2正則化項を加え過学習の影響を防ぐためのアルゴリズム


###### Q2. 過学習への対処法を４つ挙げよ。


###### Q3. ランダムフォレストのmax_depthのデフォルトはNoneとなっている。どのような決定木が生成されるか説明せよ。
Reference: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
>max_depth : integer or None, optional (default=None)  
>The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

##### Q4. sklearnのGradientBoostingRregressorのmax_depthのデフォルトを確認せよ。
Reference: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

## [Work-2-1]  
以下コードを一度よく確認してから実行し、正常終了することを確認して下さい。アンサンブル学習の強さ（R2値が相対的に高い）が確認できるはずです。もちろん、このデータだけで結論を一般化はできませんが、アンサンブル学習はその汎化能力の高さから広く活用されていることを覚えておきましょう。

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

# set dataframe
dataset = load_boston()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

# holdout
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.20,
                                                    random_state=0)
# make pipelines for modeling
pipe_ols = Pipeline([('scl',StandardScaler()),('est',LinearRegression())])
pipe_ridge = Pipeline([('scl',StandardScaler()),('est',Ridge(random_state=1))])
pipe_rf = Pipeline([('scl',StandardScaler()), ('est',RandomForestRegressor(random_state=1))])
pipe_gbr = Pipeline([('scl',StandardScaler()),('est',GradientBoostingRegressor(random_state=1))])

# build models
pipe_ols.fit(X_train, y_train)
pipe_ridge.fit(X_train, y_train)
pipe_rf.fit(X_train, y_train)
pipe_gbr.fit(X_train, y_train)

# print the performance
print('R2 score of OLS: %.6f' % r2_score(y_test, pipe_ols.predict(X_test)))
print('R2 score of Ridge: %.6f' % r2_score(y_test, pipe_ridge.predict(X_test)))
print('R2 score of RandomForest: %.6f' % r2_score(y_test, pipe_rf.predict(X_test)))
print('R2 score of GradinetBoostingRegressor: %.6f' % r2_score(y_test, pipe_gbr.predict(X_test)))

## [Work-2-2]  
以下コードを完成させ（Work2-1の再現）、正常終了させよ。

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.[----------] import StandardScaler
from sklearn.[----------] import LinearRegression, Ridge
from sklearn.[----------] import RandomForestRegressor, GradientBoostingRegressor
from sklearn.[----------] import train_test_split
from sklearn.[----------] import Pipeline
from sklearn.[----------] import r2_score

# set dataframe
dataset = load_boston()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

# holdout
X_train, [----------], y_train, [----------] = train_test_split(X, y, test_size=0.20, random_state=0)

# make pipelines for modeling
pipe_ols = Pipeline([('scl',StandardScaler()),('est',[----------])])
pipe_ridge = Pipeline([('scl',StandardScaler()),('est',[----------](random_state=1))])
pipe_rf = Pipeline([('scl',StandardScaler()),('est',[----------](random_state=1))])
pipe_gbr = Pipeline([('scl',StandardScaler()),('est',[----------](random_state=1))])

# build models
pipe_ols.[----------](X_train, y_train)
pipe_ridge.[----------](X_train, y_train)
pipe_rf.[----------](X_train, y_train)
pipe_gbr.[----------](X_train, y_train)

# print the performance
print('R2 score of OLS: %.6f' % r2_score(y_test, pipe_ols.[----------](X_test)))
print('R2 score of Ridge: %.6f' % r2_score(y_test, pipe_ridge.[----------](X_test)))
print('R2 score of RandomForest: %.6f' % r2_score(y_test, pipe_rf.[----------](X_test)))
print('R2 score of GradinetBoostingRegressor: %.6f' % r2_score(y_test, pipe_gbr.[----------](X_test)))

## [Work-2-3]  
以下のコードを完成させ（Work-2-1の再現）、正常終了させよ。

In [ ]:
# import libraries
import [----------] as pd
import [----------] as np
from sklearn.datasets import load_boston
from sklearn.[----------] import Standard[----------]
from sklearn.[----------] import Linear[----------], [----------]
from sklearn.[----------] import Random[----------], Gradient[----------]
from sklearn.[----------] import train_[----------]
from sklearn.[----------] import Pipe[----------]
from sklearn.[----------] import r2_[----------]

# set dataframe
dataset = load_boston()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

# holdout
[----------], [----------], y_train, [----------] = [----------](X, y, test_size=0.20, random_state=0)

# make pipelines for modeling
pipe_ols = Pipeline([('scl',[----------]),('est',[----------])])
pipe_ridge = Pipeline([('scl',[----------]),('est',[----------](random_state=1))])
pipe_rf = Pipeline([('scl',[----------]),('est',[----------](random_state=1))])
pipe_gbr = Pipeline([('scl',[----------]),('est',[----------](random_state=1))])

# build models
pipe_ols.[----------]([----------], [----------])
pipe_ridge.[----------]([----------], [----------])
pipe_rf.[----------]([----------], [----------])
pipe_gbr.[----------]([----------], [----------])

# print the performance
print('R2 score of OLS: %.6f' % r2_score(y_test, pipe_ols.[----------](X_test)))
print('R2 score of Ridge: %.6f' % r2_score(y_test, pipe_ridge.[----------](X_test)))
print('R2 score of RandomForest: %.6f' % r2_score(y_test, pipe_rf.[----------](X_test)))
print('R2 score of GradinetBoostingRegressor: %.6f' % r2_score(y_test, pipe_gbr.[----------](X_test)))

## [Work-2-4]  
以下のコードを完成させ（Work-2-1の再現）、正常終了させよ。

In [ ]:
# import libraries
import [----------] as pd
import [----------] as np
from sklearn.datasets import load_boston
from sklearn.[----------] import [----------]  #前処理
from sklearn.[----------] import [----------],[----------]  #線形モデル
from sklearn.[----------] import [----------],[----------]  #アンサンブルモデル
from sklearn.[----------] import [----------]  #holdout
from sklearn.[----------] import [----------]  #パイプライン
from sklearn.[----------] import [----------]  #R2スコア

# set dataframe
dataset = load_boston()
X = pd.[----------](dataset.data, columns=dataset.feature_names)
y = pd.[----------](dataset.target, name='y')

# holdout
[----------], [----------], [----------], [----------] = [----------](X,y,[----------]=0.20,random_state=0)

# make pipelines for modeling
pipe_ols = [----------]([('scl',[----------]),('est',[----------])])
pipe_ridge = [----------]([('scl',[----------]),('est',[----------](random_state=1))])
pipe_rf = [----------]([('scl',[----------]),('est',[----------](random_state=1))])
pipe_gbr = [----------]([('scl',[----------]),('est',[----------](random_state=1))])

# build models
pipe_ols.[----------]([----------], [----------])
pipe_ridge.[----------]([----------], [----------])
pipe_rf.[----------]([----------], [----------])
pipe_gbr.[----------]([----------], [----------])

# print the performance
print('R2 score of OLS: %.6f' % r2_score(y_test, pipe_ols.[----------](X_test)))
print('R2 score of Ridge: %.6f' % r2_score(y_test, pipe_ridge.[----------](X_test)))
print('R2 score of RandomForest: %.6f' % r2_score(y_test, pipe_rf.[----------](X_test)))
print('R2 score of GradinetBoostingRegressor: %.6f' % r2_score(y_test, pipe_gbr.[----------](X_test)))

## [Work-3-1]（Codeの分解）    
以下コードを実施し、正常終了することを確認せよ。データの読み込み、データ前処理、アルゴリズムの呼び出し、交叉検証準備、一連の処理を束ねる便利機能、評価指標の呼び出しと、モデル構築に必要な機能が網羅されていることを確認せよ。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

## [Work-3-2]（Codeの分解）    
サンプルデータ（dataset）の属性dataはNumpyのarray（配列型）であることを確認せよ。

In [ ]:
# set dataframe
dataset = load_boston()
dataset.data

## [Work-3-3]（Codeの分解）    
サンプルデータ（dataset）の属性feature_namesはNumpyのarray（配列型）であることを確認せよ。

In [ ]:
dataset.feature_names

## [Work-3-4]（Codeの分解）    
特徴量ベクトル（dataset.data）とカラム名（dataset.feature_names）からデータフレームが構成されることを確認せよ。

In [ ]:
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
X.head(10)

## [Work-3-5]（Codeの分解）  
正解データがdatasetオブジェクトの属性targetでNumpyのarray型で格納されていることを確認せよ。

In [ ]:
dataset.target

## [Work-3-6]（Codeの分解）  
正解データをカラム名"y"でシリーズ化していることを確認せよ。

In [ ]:
y = pd.Series(dataset.target, name='y')
y.head(10)

## [Work-3-7]（Codeの分解）  
train_test_split関数によって、Xとyが80%:20%に分割されていることを確認せよ。

In [ ]:
# cross-validation(holdout)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=0)

# print the shape
print(X.shape)
print(X_train.shape)
print(X_test.shape)

## [Work-3-8]（Codeの分解）  
パイプラインの構成要素をnamed_stepsで呼び出さることを確認せよ。  
StandardScalerは学習データの各変数（縦方向）の平均と標準偏差を記憶し標準化するためのライブラリである。

In [ ]:
# make pipelines for modeling
pipe_ridge = Pipeline([('scl',StandardScaler()),('est',Ridge())])
pipe_gbr = Pipeline([('scl',StandardScaler()),('est',GradientBoostingRegressor(random_state=1))])

# get pipeline elements
pipe_ridge.named_steps['scl']

## [Work-3-9]（Codeの分解）  
パイプラインの構成要素をnamed_stepsで呼び出さることを確認せよ。  
呼び出されたリッジ回帰のパラメータalphaが正則化項の重みであることを思い出すこと。

In [ ]:
# get pipeline elements
pipe_ridge.named_steps['est']

## [Work-3-10]（Codeの分解）  
以下のコードを実行し、パイプラインの構成要素をnamed_stepsで呼び出さることを確認せよ。呼び出された勾配ブースティングのパラメータn_estimators=100は決定木を100本直列で学習させるということである。パラメータmax_depth=3は各決定木の深さである。またrandome_stateが指定した値であることも確認せよ。

In [ ]:
# get pipeline elements
pipe_gbr.named_steps['est']

また以下パラメータについて、以下のイメージが持てるか確認しましょう。
- n_estimatorsを大きくすると過学習の危険性が増える
- max_depthを大きくすると過学習の危険性が増える
- random_stateを指定しないと同じモデルを再現できなくなる

## [Work-3-11]（Codeの分解）  
r2_score(正解ベクトル, 予測ベクトル)でR2値が取得できることを確認せよ。

In [ ]:
print(r2_score(y_test, pipe_ridge.predict(X_test)))
print(r2_score(y_test, pipe_gbr.predict(X_test)))